# Compute resource drought masks from generation thresholds

This notebook uses the previously computed daily generation thresholds to identify resource drought days in the renewable energy generation data.

The low-generation thresholds are calculated per resource, grid-cell and day-of-the-year. This notebook checks which days are below this threshold, and saves a binary mask that is 1 for low-generation (resource drought) days, and 0 for days that fall in the normal range of energy generation.

In [2]:
from renewable_data_load import *

In [ ]:
# Set data directory path
data_directory = "../../data/"

## Define functions

In [ ]:
def create_drought_mask_gwl(resource, module, simulation, reference_gwl, mask_gwl):
    
    # offshore wind data only exists in d03, otherwise calculate for WECC wide d02
    if module == "offshore":
        domain = "d03"
    else:
        domain = 'd02'

    variable = "cf"
    frequency = "day"

    print(f"Calculating drought mask for GWL: {mask_gwl}°C")
    # Get bounds for reference GWL period
    WRF_sim_name = sim_name_dict[simulation]
    model = WRF_sim_name.split("_")[1]
    ensemble_member = WRF_sim_name.split("_")[2]
    ref_start_year, ref_end_year = get_gwl_crossing_period(model, ensemble_member, mask_gwl)

    threshold_file = f"{data_directory}thresholds/{resource}_{module}_{domain}_{variable}_{simulation}_gwlref{reference_gwl}_10th_pctile.nc"
    drought_threshold_ds = xr.open_dataset(threshold_file)

    # clear encoding to avoid Zarr v3 codec issues
    for var in drought_threshold_ds.data_vars:
        drought_threshold_ds[var].encoding = {}

    ren_ds = get_ren_data_concat(resource, module, domain, variable, frequency, simulation)
    ren_ds = ren_ds.convert_calendar("noleap")
    ren_ds = ren_ds.sel(time=slice(f"{ref_start_year}-01-01",f"{ref_end_year}-12-31"))

    ## Subtract the threshold from the ren data to get drought values
    # reshape array
    ds_doy = ren_ds.copy(deep=True)
    ds_doy['dayofyear'] = ds_doy.time.dt.dayofyear
    ds_doy['year'] = ds_doy.time.dt.year
    ds_doy = ds_doy.assign_coords(
        {"dayofyear":ds_doy.time.dt.dayofyear,
        "year":ds_doy.time.dt.year})
    # reshape time dimension
    ds_doy = ds_doy.drop_vars("time").set_index(time=['dayofyear','year']).unstack()
    drought_ds = (ds_doy - drought_threshold_ds.reference_gen).load()
    # reshape back into daily timeseries
    drought_ds = drought_ds.stack(time=['year','dayofyear'])
    drought_ds = drought_ds.reset_index("time").assign_coords(time=ren_ds.time)
    drought_ds = drought_ds.load()

    # Create binary drought mask for entire time series
    # 1 where drought_ds < 0 (below threshold), 0 otherwise
    drought_mask = xr.where(drought_ds < 0, 1, 0)
    drought_mask.name = "drought_mask"

    # save into the drought mask folder
    drought_mask = drought_mask.load()

    # Save drought mask to file
    mask_output_file = f"{data_directory}drought_masks/{resource}_{module}_{domain}_{variable}_{simulation}_gwl{mask_gwl}_drought_mask_only.zarr"

    # Add metadata to drought mask
    drought_mask.attrs['resource'] = resource
    drought_mask.attrs['module'] = module
    drought_mask.attrs['domain'] = domain
    drought_mask.attrs['variable'] = variable
    drought_mask.attrs['simulation'] = simulation
    drought_mask.attrs['reference_gwl'] = float(reference_gwl)
    drought_mask.attrs['description'] = 'Binary drought mask: 1 = drought (below threshold), 0 = no drought'

    # Clear encoding from data variable and all coordinates to avoid codec conflicts
    drought_mask.encoding = {}
    for coord in drought_mask.coords:
        drought_mask.coords[coord].encoding = {}
    
    # Save with Zarr v3 - let xarray choose v3-compatible codecs
    # Only specify dtype, let compression use v3 defaults
    encoding = {'drought_mask': {'dtype': 'int32'}}
    drought_mask.to_zarr(mask_output_file, encoding=encoding, mode='w', consolidated=True)

In [ ]:
# compute mask for a single case
resource = "windpower"
module = "onshore"
reference_gwl = 0.8
mask_gwl = 2.0
simulation = "miroc6"

create_drought_mask_gwl(resource, module, simulation, reference_gwl, mask_gwl)

Calculating drought mask for GWL: 2.0°C


/Users/work/miniconda3/envs/renewables-analysis/lib/python3.12/site-packages/zarr/api/asynchronous.py:244: ZarrUserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


In [ ]:
# compute resource drought mask for all resource, module, and GWL level combinations

resource = "windpower"
module = "onshore"

# resource = "windpower"
# module = "offshore"

# resource = "pv"
# module = "utility"

reference_gwl = 0.8

for simulation in ["mpi-esm1-2-hr","taiesm1", 'ec-earth3',"miroc6"]:
    for mask_gwl in [0.8,2.0,]:
        print(f"Processing simulation: {simulation}, mask GWL: {mask_gwl}°C")
        create_drought_mask_gwl(resource, module, simulation, reference_gwl, mask_gwl)

Processing simulation: mpi-esm1-2-hr, mask GWL: 0.8°C
Calculating drought mask for GWL: 0.8°C


/Users/work/miniconda3/envs/renewables-analysis/lib/python3.12/site-packages/zarr/api/asynchronous.py:244: ZarrUserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


Processing simulation: mpi-esm1-2-hr, mask GWL: 2.0°C
Calculating drought mask for GWL: 2.0°C


/Users/work/miniconda3/envs/renewables-analysis/lib/python3.12/site-packages/zarr/api/asynchronous.py:244: ZarrUserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


Processing simulation: taiesm1, mask GWL: 0.8°C
Calculating drought mask for GWL: 0.8°C


...

/Users/work/miniconda3/envs/renewables-analysis/lib/python3.12/site-packages/zarr/api/asynchronous.py:244: ZarrUserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


Processing simulation: miroc6, mask GWL: 0.8°C
Calculating drought mask for GWL: 0.8°C


/Users/work/miniconda3/envs/renewables-analysis/lib/python3.12/site-packages/zarr/api/asynchronous.py:244: ZarrUserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


Processing simulation: miroc6, mask GWL: 2.0°C
Calculating drought mask for GWL: 2.0°C


/Users/work/miniconda3/envs/renewables-analysis/lib/python3.12/site-packages/zarr/api/asynchronous.py:244: ZarrUserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


In [ ]:
def create_drought_mask_timeseries(resource, module, simulation, reference_gwl):
    if module == "offshore":
        domain = "d03"
    else:
        domain = 'd02'

    variable = "cf"
    frequency = "day"

    print(f"Calculating drought mask for full timeseries")

    threshold_file = f"{data_directory}thresholds/{resource}_{module}_{domain}_{variable}_{simulation}_gwlref{reference_gwl}_10th_pctile.nc"
    drought_threshold_ds = xr.open_dataset(threshold_file)

    # clear encoding to avoid Zarr v3 codec issues
    for var in drought_threshold_ds.data_vars:
        drought_threshold_ds[var].encoding = {}

    ren_ds = get_ren_data_concat(resource, module, domain, variable, frequency, simulation)
    ren_ds = ren_ds.convert_calendar("noleap")

    ## Subtract the threshold from the ren data to get drought values
    # reshape array
    ds_doy = ren_ds.copy(deep=True)
    ds_doy['dayofyear'] = ds_doy.time.dt.dayofyear
    ds_doy['year'] = ds_doy.time.dt.year
    ds_doy = ds_doy.assign_coords(
        {"dayofyear":ds_doy.time.dt.dayofyear,
        "year":ds_doy.time.dt.year})
    # reshape time dimension
    ds_doy = ds_doy.drop_vars("time").set_index(time=['dayofyear','year']).unstack()
    drought_ds = (ds_doy - drought_threshold_ds.reference_gen).load()
    # reshape back into daily timeseries
    drought_ds = drought_ds.stack(time=['year','dayofyear'])
    drought_ds = drought_ds.reset_index("time").assign_coords(time=ren_ds.time)
    drought_ds = drought_ds.load()

    # Create binary drought mask for entire time series
    # 1 where drought_ds < 0 (below threshold), 0 otherwise
    drought_mask = xr.where(drought_ds < 0, 1, 0)
    drought_mask.name = "drought_mask"

    # save into the drought mask folder
    drought_mask = drought_mask.load()

    # Save drought mask to file
    mask_output_file = f"{data_directory}drought_masks/{resource}_{module}_{domain}_{variable}_{simulation}_timeseries_drought_mask_only.zarr"

    # Add metadata to drought mask
    drought_mask.attrs['resource'] = resource
    drought_mask.attrs['module'] = module
    drought_mask.attrs['domain'] = domain
    drought_mask.attrs['variable'] = variable
    drought_mask.attrs['simulation'] = simulation
    drought_mask.attrs['reference_gwl'] = float(reference_gwl)
    drought_mask.attrs['description'] = 'Binary drought mask: 1 = drought (below threshold), 0 = no drought'

    # Clear encoding from data variable and all coordinates to avoid codec conflicts
    drought_mask.encoding = {}
    for coord in drought_mask.coords:
        drought_mask.coords[coord].encoding = {}
    
    # Save with Zarr v3 - let xarray choose v3-compatible codecs
    # Only specify dtype, let compression use v3 defaults
    encoding = {'drought_mask': {'dtype': 'int32'}}
    drought_mask.to_zarr(mask_output_file, encoding=encoding, mode='w', consolidated=True)

In [7]:
# compute mask for a single case
resource = "pv"
module = "utility"
reference_gwl = 0.8
simulation = "miroc6"

create_drought_mask_timeseries(resource, module, simulation, reference_gwl)

Calculating drought mask for full timeseries


In [ ]:
# compute resource drought mask for all resource, module, and GWL level combinations

resource = "windpower"
module = "onshore"

# resource = "windpower"
# module = "offshore"

# resource = "pv"
# module = "utility"

reference_gwl = 0.8

for simulation in ["mpi-esm1-2-hr","taiesm1", 'ec-earth3',"miroc6"]:
    for mask_gwl in [0.8,2.0,]:
        print(f"Processing simulation: {simulation}, mask GWL: {mask_gwl}°C")
        create_drought_mask_timeseries(resource, module, simulation, reference_gwl, mask_gwl)